### 读取数据库中的所有表名

In [6]:
import pymysql.cursors 
 
 
dbname = 'db_eastmoney'
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='0000',
                             db=dbname,
                             charset='utf8',
                             cursorclass=pymysql.cursors.DictCursor)
results =[]
try:
    with connection.cursor() as cursor:
        sql = '''SHOW TABLES'''
        cursor.execute(sql)
        result = cursor.fetchall() 
        for i in range(len(result)):
            results.append(result[i]['Tables_in_'+dbname])
finally:
    connection.close()

print(results)

['czce_ap', 'czce_cf', 'czce_cy', 'czce_fg', 'czce_jr', 'czce_lr', 'czce_ma', 'czce_oi', 'czce_pm', 'czce_ri', 'czce_rm', 'czce_rs', 'czce_sf', 'czce_sm', 'czce_sr', 'czce_ta', 'czce_wh', 'czce_zc', 'dce_a', 'dce_b', 'dce_bb', 'dce_c', 'dce_cs', 'dce_eb', 'dce_eg', 'dce_fb', 'dce_i', 'dce_j', 'dce_jd', 'dce_jm', 'dce_l', 'dce_m', 'dce_p', 'dce_pp', 'dce_rr', 'dce_v', 'dce_y', 'shfe_ag', 'shfe_al', 'shfe_au', 'shfe_bu', 'shfe_cu', 'shfe_fu', 'shfe_hc', 'shfe_ni', 'shfe_pb', 'shfe_rb', 'shfe_ru', 'shfe_sn', 'shfe_sp', 'shfe_ss', 'shfe_wr', 'shfe_zn']


In [61]:
import pymysql
import pandas as pd
import re
import itertools
 
abitrage_dic = {}
conn = pymysql.connect(host = "localhost",port=3306,user='root',passwd='0000',db='db_eastmoney',charset="utf8")
for i in results:
    sql_query = 'SELECT * FROM '+i
    df1 = pd.read_sql(sql_query, con=conn)

#     print(df1)
    
    df1.name = df1.name.apply(lambda x:eval(repr(x).replace('\\\\', '\\')))
#     print(df1.loc[0,'name'])
#     if '棉纱' not in df1.loc[0,'name']:
#         continue
    if 'TA' in df1.loc[0,'name']:
        name = ''.join(re.findall(r'[A-Za-z]', df1.loc[0,'name']))
#         name = ''.join(re.findall(u"[\u4e00-\u9fa5]+", df1.loc[0,'name']))
    else:
        name = ''.join(re.findall(u"[\u4e00-\u9fa5]+", df1.loc[0,'name']))
    df1 = df1[(df1.name.str.contains('主力')==False) & (df1.new_p!='-')]
    df1['new_p'] = df1['new_p'].astype('float64')
    df1['volume'] = df1['volume'].astype('int64')
    df1['money'] = df1['new_p']*df1['volume']
    df1 = df1[df1['money']>2000*10000]
    
    
#     print(df1)
    if len(df1)>2:
        
        cate_list = df1.code.tolist()
        abitrage_dic[name] = list(itertools.combinations(cate_list, 2))
        print(name,list(itertools.combinations(cate_list, 2)))
conn.close()  #使用完后记得关掉
# print(abitrage_dic)

苹果 [('AP110', 'AP111'), ('AP110', 'AP201'), ('AP111', 'AP201')]
一号棉花 [('CF107', 'CF109'), ('CF107', 'CF111'), ('CF107', 'CF201'), ('CF109', 'CF111'), ('CF109', 'CF201'), ('CF111', 'CF201')]
玻璃 [('FG108', 'FG109'), ('FG108', 'FG201'), ('FG109', 'FG201')]
甲醇 [('MA107', 'MA108'), ('MA107', 'MA109'), ('MA107', 'MA110'), ('MA107', 'MA201'), ('MA108', 'MA109'), ('MA108', 'MA110'), ('MA108', 'MA201'), ('MA109', 'MA110'), ('MA109', 'MA201'), ('MA110', 'MA201')]
菜油 [('OI109', 'OI111'), ('OI109', 'OI201'), ('OI111', 'OI201')]
菜粕 [('RM108', 'RM109'), ('RM108', 'RM111'), ('RM108', 'RM201'), ('RM109', 'RM111'), ('RM109', 'RM201'), ('RM111', 'RM201')]
硅铁 [('SF107', 'SF108'), ('SF107', 'SF109'), ('SF107', 'SF110'), ('SF107', 'SF201'), ('SF108', 'SF109'), ('SF108', 'SF110'), ('SF108', 'SF201'), ('SF109', 'SF110'), ('SF109', 'SF201'), ('SF110', 'SF201')]
锰硅 [('SM107', 'SM108'), ('SM107', 'SM109'), ('SM107', 'SM110'), ('SM107', 'SM201'), ('SM108', 'SM109'), ('SM108', 'SM110'), ('SM108', 'SM201'), ('SM10

In [ ]:
s = ['苹果',"一号棉花",'玻璃','甲醇','菜粕','硅铁','锰硅','白糖','豆一','豆二','玉米','玉米淀粉',]

In [65]:
import akshare as ak
futures_contract_detail_df = ak.futures_contract_detail(contract='AP2110')
print(futures_contract_detail_df)

       item                                      value
0      交易品种                                        鲜苹果
1    最小变动价位                                       1元/吨
2      交易时间  上午 09:00-10:15 10:30-11:30 下午 13:30-15:00
3      交割品级                            郑州商品交易所苹果交割质量标准
4      交割方式                                       实物交割
5      交易单位                                      10吨/手
6    涨跌停板幅度                               上一交易日结算价的±5％
7     最后交易日                             合约交割月份的第10个交易日
8   最低交易保证金                                    合约价值的7%
9      交易代码                                         AP
10     报价单位                                   元（人民币）/吨
11   合约交割月份                          1、3、5、7、10、11、12月
12    最后交割日            仓单交割：合约交割月份的第12个交易日 车（船）板交割：合约�
13    交易手续费                        5.00 元/手,平今仓5.00元/手
14    上市交易所                                    郑州商品交易所


In [68]:
from bs4 import BeautifulSoup
import urllib.parse
def askURL(url):
    head = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36 Edg/89.0.774.63"}
    req = urllib.request.Request(url = url,headers=head)
    html = ""
    try:
        response = urllib.request.urlopen(req)
        html = response.read().decode('utf8')
        return html
    except urllib.error.URLError as e:
        if hasattr(e,'code'):
            print(e.code)
        if hasattr(e,'reason'):
            print(e.reason)
            
    


url = 'https://www.akshare.xyz/zh_CN/latest/data/futures/futures.html#id5'
url = 'http://www.shfe.com.cn/products/cu/'
html = askURL(url)
bs = BeautifulSoup(html,"html.parser")
# data = pd.read_html(url,encoding='utf-8')
bs

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta charset="utf-8"/>
<title>上市品种</title>
<link href="favicon.ico" rel="icon"/>
<meta content=" " name="keywords"/>
<meta content=" " name="description"/>
<!-- 全局风格CSS -->
<link href="/styles/base.css" rel="stylesheet" type="text/css"/>
<link href="/styles/head.css" rel="stylesheet" type="text/css"/>
<link href="/styles/list.css" rel="stylesheet" type="text/css"/>
<link href="/styles/nav_cn.css" rel="stylesheet" type="text/css"/>
<!-- 核心JS加载 -->
<script src="/js/jquery.js" type="text/javascript"></script>
<script src="/js/instrument_leftmenu.js" type="text/javascript"></script>
<script src="/js/mainmenu.js" type="text/javascript"></script>
<script src="/js/footer.js" type="text/javascript"></script>
<script src="/js/products_delayed_market.js" type="text/javascript"></script>
<script src="/js/products_timeprice.j